In [6]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [7]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO("best_predictions.json")
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

NameError: name 'output' is not defined

In [ ]:
coco = COCO("best_predictions.json")
img_ids = coco.getImgIds()

In [ ]:
gt = []

coco = COCO("best_predictions.json")

'''
image id 별로 GT 값 추가 (여기서는 test.json을 사용하였기 때문에 GT가 없습니다.)
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                    float(annotation['score']),
                    float(annotation['bbox'][0]),
                    float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                    float(annotation['bbox'][1]),
                    (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

In [1]:
import pandas as pd

In [2]:
pd.read_json("./best_predictions.json")

ValueError: Unexpected character found when decoding array value (2)

In [12]:
coco = COCO("./best_predictions.json")
image_info = coco.loadImgs(coco.getImgIds()[0])[0]

annotation_id = coco.getAnnIds(imgIds=image_info['id'])
annotation_info_list = coco.loadAnns(annotation_id)

loading annotations into memory...


JSONDecodeError: Expecting ',' delimiter: line 1 column 16515073 (char 16515072)